In [1]:
import pickle
import pandas as pd
import numpy as np
import scipy
from scipy import stats
import collections
from collections import defaultdict
import random
import copy
import math
import itertools
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable
import time
import pulp
# add paths
# path_to_cplex = r'/opt/anaconda3/lib/python3.8/site-packages/pulp/solverdir/cbc/osx/64/cbc'
# solver = pulp.CPLEX_CMD(path=path_to_cplex)
from collections import Counter
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update({'font.size': 14})
mpl.rc('text', usetex = False)
mpl.rc('font', family = 'serif')
import networkx as nx


# 1. Load input

Load episode set $E$ with the users that retweeted each original tweet in the trace. 

Each episode $E_{s}$ includes the users that retweeted s, ordered chronologically, as they appear in the trace. The first user in each episode is the user that originally tweeted the tweet, and is denoted by $r_{s}$. Subsequent users in $E_{s}$ are users that retweeted s, either directly from user $r_{s}$ or from another user that appears in Es before them.

In [2]:
lines = 224667
dataset = 'covid'
# random.seed(10)

In [3]:
E = pickle.load(open("./Extracted/"+dataset+"/E"+ str(lines) + ".p", "rb"))

In [4]:
for tweet in E:
    E[tweet] = list(dict.fromkeys(E[tweet]))

Load episode set $D$ with the users that retweeted each original tweet in the trace. 

In [5]:
D = pickle.load(open("./Extracted/"+dataset+"/D"+ str(lines) + ".p", "rb"))

Load the set of original tweets denoted by $S$. 

The set of original tweets is denoted by $S$, where $S$ = S is the total number of original tweets

In [6]:
S = pickle.load(open("./Extracted/"+dataset+"/S"+ str(lines) + ".p", "rb"))

Load $U$ set xith unique users

In [7]:
U = pickle.load(open("./Extracted/"+dataset+"/U"+ str(lines) + ".p", "rb"))
U = list(U)

Load $M_{ij}$ variables that count number of episodes where the ordered pair (i,j) appears

In [8]:
M = pickle.load(open("./Extracted/"+dataset+"/M_d"+ str(lines) + ".p", "rb"))

# 2. Find important quantities

- Number of unique users N
- Number of episodes / original tweets S
- Number of active pairs (i,j)

For every active pair we have to count the $\sigma_{ij}$'s

In [9]:
N = len(U)
print('Number of unique users N =', N)

Number of unique users N = 44602


In [10]:
print('Number of Episodes (original tweets) S =',len(E))

Number of Episodes (original tweets) S = 9722


In [11]:
active_pairs_n = 0 
for i in M:
    active_pairs_n+=len(M[i])

In [12]:
print('Number of active user pairs in the trace:', active_pairs_n, 'out of the', N*(N-1), 'possible pairs')

# 50000 lines: Number of active user pairs in the trace: 415297 out of the 47837972 possible pairs
# 70000 lines: Number of active user pairs in the trace: 818995 out of the 90069590 possible pairs

Number of active user pairs in the trace: 41020780 out of the 1989293802 possible pairs


# 3. CONSTRAINED-EM Algorithm

Define important functions 

In [13]:
def f_check(E, Q):
    '''
    Function that checks feasibility of results

    '''
    retweets = 0 # minimum existing edges
    total_feasible_edges = []
    total_inf = 0 
    for s in E:
        feasible_edges = 0 
        for j in E[s]:
            indx = E[s].index(j)
            if indx!=0:
                u_before = E[s][:indx]
                for i in u_before: 
                    if j in Q[i] and Q[i][j] > 0.5:
                        feasible_edges +=1
                        total_feasible_edges.append((i,j))

        infeasible = (len(E[s]) - 1) - feasible_edges
        if infeasible > 0:
            print('Tweet', s, 'retweeted by', len(E[s])-1, 'users in total. But, we only found:', feasible_edges, 'feasible edges, so the infeasible ones are:', infeasible)            
            total_inf+=infeasible
            
        retweets += len(E[s])-1
        total_feasible_edges = list(set(total_feasible_edges))
        
    print('Total feasbile edges:', len(total_feasible_edges), 'Number of retweets', retweets)
    print('Total infeasible edges:', total_inf)

In [14]:
def flatten(obj):
    if type(obj) == list:
        return [l for L in obj for l in L]
    if type(obj) == dict:
        return [l for i in obj for l in obj[i].values()]
    if type(obj) == defaultdict:
        return [l for i in obj for l in obj[i].values()]

In [15]:
def update_Q(M, Q, s, a, b, r):
    for i in M:
        for j in M[i]:
            e = M[i][j] * s[i][j]
            numer = r * (a**e) * ((1-a)**(M[i][j]-e))
            denom = r * (a**e) * ((1-a)**(M[i][j]-e)) + (1-r) * (b**e) * ((1-b)**(M[i][j]-e))
            Q[i][j] = numer/denom   
    return Q

In [16]:
def update_a(M, Q, s):
    a = ((sum([M[i][j] * Q[i][j] * s[i][j]  for i in M for j in M[i]]))
        /((sum([M[i][j] * Q[i][j] for i in M for j in M[i]])))) 
    return(a)

In [17]:
def update_b(M, Q, s):
    b = ((sum([M[i][j] * (1-Q[i][j]) * s[i][j]  for i in M for j in M[i]]))
        /((sum([M[i][j] * (1-Q[i][j]) for i in M for j in M[i]]))))
    return(b)

In [18]:
def update_r(Q, N, active_pairs_n):
#     r = (sum([Q[i][j] for i in Q for j in Q[i]]))/(N*(N-1)) 
    r = (sum([Q[i][j] for i in Q for j in Q[i]]))/(active_pairs_n)      
    return(r)

In [19]:
def update_s(model, M, Q, x, a, b, s, active_pairs, lam):
    """ 
    This function updates the s_{ij} parameters of the optimization problemm

    Parameters
    ----------
    M : dict
        Dictionary with M_{ij} values tha show how many times an ordered pair (i,j) appears in the trace.
    Q : dict
        Dictionary with the Q_{ij} values
    W : dict
        Dictionary with the coefficients of the problem 
    x : dict
        Dictionary with decision variables
    active_pairs : list
        Active pairs of the problem
    s : dict
        Dictionary with the updated s_{ij} parameters
        
    Returns
    ----------
    s : dict
        Dictionary with the updated s_{ij} parameters
    """
    
    W = defaultdict(dict)
#     W_ones = defaultdict(dict)

#     start = time.time()
    for pair in active_pairs:
        i = pair[0]
        j = pair[1]
        W[i][j] = M[i][j]*((Q[i][j]*math.log(a/(1-a)))+ (1-Q[i][j])*math.log(b/(1-b))) + random.uniform(0,0.001)

#     minv = min((set(min(list((i.values())) for i in W.values()))))
#     print('min W:', minv)
    
        
#     end = time.time()
#     print('(s) W:', end-start)
#     for i in M:
#         for j in M[i]:
#             if (i,j) in active_pairs:
#                 W[i][j] = M[i][j]*((Q[i][j]*math.log(a/(1-a)))+ (1-Q[i][j])*math.log(b/(1-b))) + random.uniform(0,0.001)
#             else: # s[i][j] = 1
#                 W_ones[i][j] = M[i][j]*((Q[i][j]*math.log(a/(1-a)))+ (1-Q[i][j])*math.log(b/(1-b))) + random.uniform(0,0.001)
    
#     start = time.time()
    s = pulp_solve(model, active_pairs, W, x, s, lam)
#     end = time.time()
#     print('(s) pulp_solve:', end-start)
    return(s, W)

In [20]:
def pulp_create_reduct(E, M):
    """ 
    This function initializes the optimization problem with reduced constraints. Runs the first time

    Returns
    ----------
    model : LpProblem
        Pulp model of the problem 
    x : Dict
        Dictionary with decision variables
    """
    
    
    # Initialize the maximization problem
    model = LpProblem(name="constr-newman", sense=LpMaximize)
    
    # For each episode in $E$, each line in constraints_list includes:
    # - for each user j in $E$, the users before them in pos 0 and 
    # - the user itself in pos 1
    
    sij = defaultdict(dict)
    constraints_list = []
    
    for s in D:
        for users_list in list(D[s].values()):
            index_now = list(D[s].values()).index(users_list)
            if index_now == 0:
                for u in list(D[s].values())[1]:
                    sij[D[s][0][0]][u] = 1
            else:
                for u in users_list:
                    u_before_l = list(D[s].values())[:index_now]
                    u_before = [item for sublist in u_before_l for item in sublist]
                    if u in u_before: u_before.remove(u)
                    constraints_list.append([u_before, u])

    len_b = len(constraints_list)
    print('constraints before', len(constraints_list))

    # phase I: delete all constraints that include a pair with sij = 1
    
    for c in list(constraints_list):
        j = c[1]
        for i in c[0]:
            if i in sij and j in sij[i] and sij[i][j]==1:
                constraints_list.remove(c)
                break
                
    # For each episode in $E$, the constraints_dictionary includes:
    # - the user j as keys
    # - the users that come before them for each constraint that they appear in pos j as values
    
    constraints_list.sort()
    constraints_list = list(constraints_list for constraints_list,_ in itertools.groupby(constraints_list))
    
#     print('3rd:', [[924604094999908352, 952286580890365952, 367664497],787929838774517760] in constraints_list)
    
    constraints_dict = dict()
    ind = 0 
    for c in constraints_list:
            j = c[1]
            if j in constraints_dict:
                constraints_dict[j].append([c[0],ind])
            else:
                constraints_dict[j] = []
                constraints_dict[j].append([c[0],ind])
            ind+=1

    # 2nd phase ----------------------------
#     print('3rdb:', [[924604094999908352, 952286580890365952, 367664497],787929838774517760] in constraints_list)

    
    for j in constraints_dict:
        for constraint1 in constraints_dict[j]:
            for constraint2 in constraints_dict[j]:
                if constraint1[1]!=constraint2[1]:
                    if set(constraint1[0]).issubset(constraint2[0]):
                        if constraint1[0]!=constraint2[0] and collections.Counter(constraint1[0]) != collections.Counter(constraint2[0]): 
                            if [constraint2[0],j] in constraints_list: # we may have already deleted it
                                constraints_list.remove([constraint2[0],j])
                    
#     print('4th:', [[924604094999908352, 952286580890365952, 367664497],787929838774517760] in constraints_list)
                            

    len_a = len(constraints_list)          
    
    print('constraints after', len(constraints_list))
    print('change of constraints:', abs(len_a - len_b))
    
    # Add decision variables 
    
    # Create active pairs and sij's randomly 
    active_pairs = []
    for i in M:
        for j in M[i]:
            if i not in sij or j not in sij[i]:
                active_pairs.append((i,j))
                sij[i][j] = random.uniform(0, 1)

    active_pairs = list(set(active_pairs))
        
    x = dict()
    x = {f"{pair[0]}-{pair[1]}":LpVariable(name=f"x{pair[0]}-{pair[1]}", lowBound=0, upBound=0.9999999999999) for pair in active_pairs}
    
    # Add constraints in model
    for c in constraints_list: 
        j = c[1]
        tmp = 0
        for i in c[0]:
            # x[k] = LpVariable(name=f"x{u}-{j}", lowBound=0, upBound=1)
            tmp+=x[f"{i}-{j}"]
        model += (tmp >= 1)
        
    print('Active pairs:', len(active_pairs))
        
    return model, x, active_pairs, sij

In [21]:
def pulp_solve(model, active_pairs, W, x, s, lam):
    """ 
    This function solves the optimization problem with PULP.
    
    Parameters
    ----------
    model : PulpModel
        Model initialized from pulp
    active_pairs : list
        Active pairs of the problem
    W : dict
        Dictionary with the coefficients of the problem 
    x : dict
        Dictionary with decision variables
    lam : float
        Lambda value by which we decreace the W's
    
    Returns
    ----------
    s : dict
        Dictionary with found s_{ij} parameters
    
    """
    
    maxv = max((set(max(list(i.values()) for i in W.values()))))
    c = maxv + 0.0001
    
    # Update the objective function
    model += lpSum((W[i][j]-c)*x[f"{i}-{j}"] for i in W for j in W[i])
#     print(model)
#     start = time.time()
    # And solve it 
    status = model.solve()
#     end = time.time()
#     print('(s) model.solve:', end-start)
    # print(f"status: {model.status}, {LpStatus[model.status]}")

#     start = time.time()
    for var in x.values():
        i = var.name.split("_")[0]
        i = int(i.split("x")[1])
        j = int(var.name.split("_")[1])
        s[i][j] = var.value()
#     end = time.time()
#     print('(s) assign to sij:', end-start)        
#     print('status:', LpStatus[model.status])

#     costp = pulp.value(model.objective)
#     print('Max objective function:', costp)
    
    return s
                    

In [22]:
def newman(eps, N, M, active_pairs_n, lam):
        """ 
        This function is the main algorithm for path inference with constraints. 

        Parameters
        ----------
        rep : int 
            Number of times to run the algorithm.
        N : int
            Number of users in the trace.
        M : dict
            Dictionary with M_{ij} values tha show how many times an ordered pair (i,j) appears in the trace.

        Returns
        ----------

        """

        iterat = 1
        # ======================== INITIALIZE ========================
        print('Initialize...')

        # 0.1 -- Initialize Q dictionary 

        Q = defaultdict(dict)

        # 0.2 -- Initialize a, b, r $ parameters
        a = random.uniform(0.5, 1)
        b = random.uniform(0, 0.5)
        r = random.uniform(0, 1)

#         a= 0.9999 
#         b= 0.0001 
#         r= 0.003

        print('Initial a=', a,'b=', b, 'r=', r)

        print('Create pulp model with reduced constraints and variables...')
        # Create pulp model, one time
        model, x, active_pairs, s = pulp_create_reduct(E, M)
        
        # ======================== START algorithm ========================

        while True: # Repeat until convergence
            # Step 1 ==== UPDATE VALUES ====

            # 1.1 -- Update Q dictionary
            
            # print('Update Q...')
#             start = time.time()
            Q = update_Q(M, Q, s, a, b, r)
#             end = time.time()
#             print('Q', start-end)
            # 1.2 -- Update a,b,r,s_{ij} parameters according to Equations
            
            # print('Update a...')
#             start = time.time()
            a = update_a(M, Q, s)
#             end = time.time()
#             print('a', start-end)
            
            # print('Update b...')
#             start = time.time()
            b = update_b(M, Q, s)
#             end = time.time()
#             print('b', start-end)
            
            # print('Update r...')
#             start = time.time()
            r = update_r(Q, N, active_pairs_n)
#             end = time.time()
#             print('r:', start-end)            
            if a ==1: 
                a = 0.9999999
                flag = True
            if b ==0: 
                b = 0.0001
                flag = True
            
            # print('Update s...')
#             start = time.time()
            s, W  = update_s(model, M, Q, x, a, b, s, active_pairs, lam)
#             end = time.time()
#             print('s:', start-end)
            # Step 2 ==== CHECK CONVERGENCE ====
            
            # print('Check convergence...')

            if iterat > 1:
                new_q = np.array(flatten(Q))
                new_a = a
                new_b = b
                
                cost = sum(s[i][j]*M[i][j]*(Q[i][j]*math.log(a/(1-a))+(1-Q[i][j])*math.log(b/(1-b))) for i in s for j in s[i])
                print('Objective cost:', cost)
                new_cost = cost
                
                changeq = np.linalg.norm(new_q - old_q)  
                changea = abs(new_a-old_a)
                changeb = abs(new_b-old_b)
                changecost = abs(new_cost - old_cost)

#                 if changeq < eps and changea < eps and changeb < eps or flag == True: 
#                 if changeq < eps or flag == True:
                if changeq<eps:
                    print('-----')
#                     print('flag', flag)
                    print('- Change q:', changeq, 'Change a:', changea, 'Change b:', changeb)
                    print('- Changecost:', changecost)
                    print('- a=', a,'b=',b,'r=', r)
                    print('cost:', cost)
                
                    Q_old = copy.deepcopy(Q)
                    old_q = np.array(flatten(Q_old))
                    old_a = a
                    old_b = b
                    old_cost = cost        
                    break
                else: 
                    print('-----')
                    print('- Change q:', changeq, 'Change a:', changea, 'Change b:', changeb)
                    print('- Changecost:', changecost)
                    print('- a=', a,'b=',b,'r=', r)
                    print('cost:', cost)
                
                    Q_old = copy.deepcopy(Q)
                    old_q = np.array(flatten(Q_old))
                    old_a = a
                    old_b = b
                    old_cost = cost
                    
                    iterat += 1
                    
            if iterat == 1:
                flag = False
                old_q = np.array(flatten(Q))
                old_a = a
                old_b = b
                
                cost = sum(s[i][j]*M[i][j]*(Q[i][j]*math.log(a/(1-a))+(1-Q[i][j])*math.log(b/(1-b))) for i in s for j in s[i])
#                 print('Objective cost:', cost)
                old_cost = cost
                
                iterat+=1
                
        # ======================== END algorithm ========================
    
        return iterat, W, a, b, r, s, Q, cost

In [ ]:
start = time.time()
random.seed(10)
print('------------------SEED:', i)
eps = 0.001
lam = 1
iterat, W, a, b, r, s, Q, cost = newman(eps, N, M, active_pairs_n, lam)
end = time.time()
print('Time:', end-start)

------------------SEED: 1197612772160344064
Initialize...
Initial a= 0.7857012973449568 b= 0.2144445273375573 r= 0.5780913011344704
Create pulp model with reduced constraints and variables...
constraints before 60675
constraints after 49916
change of constraints: 10759
Active pairs: 41011483


In [104]:
iterat 

496

In [103]:
# np.linalg.norm(np.array(flatten(Q_10))-np.array(flatten(Q_20)))
exists = 0
n_exists = 0
same = 0 
n_same = 0
for i in Q_30:
    for j in Q_30[i]:
        if i in Q_40 and j in Q_40[i]: 
            exists+=1
            if Q_30[i][j] > 0.9:
                if Q_40[i][j] > 0.9:
                    same +=1
                else:
                    n_same +=1
            else:
                if Q_40[i][j] > 0.9:
                    n_same +=1
                else:
                    same +=1
        else: 
            n_exists+=1
exists, n_exists, same, n_same

NameError: name 'Q_30' is not defined

In [26]:
print('- a=', a, '1-a', 1-a)
print('- b=', b, '1-b:', 1-b)
print('- r=', r)
print(a,b,r)
print('Feasibility check:')
f_check(E,Q)

- a= 0.9996750478190183 1-a 0.0003249521809817191
- b= 3.7354632604484254e-06 1-b: 0.9999962645367395
- r= 0.02690363162162662
0.9996750478190183 3.7354632604484254e-06 0.02690363162162662
Feasibility check:
Total feasbile edges: 11140 Number of retweets 11669
Total infeasible edges: 0


In [27]:
try: 
    pickle.dump(Q, open("./Extracted/AlgorithmResults/Q_constrained_" + str(lines) + ".p", "wb"))
except: 
    print("Unable to write to file")

In [28]:
try: 
    pickle.dump(s, open("./Extracted/AlgorithmResults/s_constrained_" + str(lines) + ".p", "wb"))
except: 
    print("Unable to write to file")

In [29]:
# list((set(max(list(i.values()) for i in W.values()))))[0]
# max((set(max(list(i.values()) for i in W.values()))))

# maxv = max((set(max(list(i.values()) for i in M.values()))))
